In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact_manual

In [2]:
def calculate_record_counts(df, frequency):
    """
    Calculate the number of records based on the given frequency.
    Weekly frequency starts every Monday, Monthly on the 1st of each month, 
    and Quarterly starting from the 1st of January, April, July, and October.
    """
    # Ensure 'Date' is a datetime type
    df['Date'] = pd.to_datetime(df['Date'])

    # Group by 'Unit', 'Type', and resample based on frequency, then calculate the count of records
    if frequency == 'Weekly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('W-MON').size().reset_index(name='Record Count')
    elif frequency == 'Monthly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('MS').size().reset_index(name='Record Count')
    elif frequency == 'Quarterly':
        resampled_df = df.set_index('Date').groupby(['Unit', 'Type']).resample('QS-JAN').size().reset_index(name='Record Count')

    return resampled_df


In [3]:
def plot_run_charts(df, frequency):
    """
    Plot run charts for each unit and an overall combined chart.
    The combined chart will display the average scores aggregated from all units for 'Patient' and 'Visitor'.
    """
    # Calculate average scores based on frequency
    record_counts = calculate_record_counts(df, frequency)

    # List of unique units
    units = df['Unit'].unique().tolist()

    # Plot for each unit
    for unit in units:
        plot_individual_run_chart(unit, record_counts, frequency)

    # Plot the combined chart
    plot_combined_run_chart(record_counts, frequency)

def plot_individual_run_chart(unit, record_counts, frequency):
    """
    Plot run chart for an individual unit.
    """
    plt.figure(figsize=(12, 6))
    plt.title(f"Run Chart for {unit} - {frequency} Frequency")
    
    # Filtering data for the current unit
    unit_df = record_counts[record_counts['Unit'] == unit]

    # Plotting logic for individual unit
    plot_run_chart_logic(unit_df)

def plot_combined_run_chart(record_counts, frequency):
    """
    Plot the combined run chart for all units.
    """
    plt.figure(figsize=(12, 6))
    plt.title(f"Combined Run Chart for All Units - {frequency} Frequency")

    # Calculating the overall average for 'Patient' and 'Visitor' across all units
    combined_df = record_counts.groupby(['Date', 'Type'])['Record Count'].sum().reset_index()

    # Plotting logic for combined chart
    plot_run_chart_logic(combined_df)

def plot_run_chart_logic(df):
    """
    Shared plotting logic for run charts.
    """
    # Creating separate dataframes for 'Patient' and 'Visitor'
    patient_df = df[df['Type'] == 'Patient']
    visitor_df = df[df['Type'] == 'Visitor']

    patient_color = '#00727C'
    visitor_color = '#9ED9D1'
    
    # Plotting 'Patient' scores
    plt.plot(patient_df['Date'], patient_df['Record Count'], label='Patient', marker='o', linestyle='-', color=patient_color)

    # Plotting 'Visitor' scores if available
    if not visitor_df.empty:
        plt.plot(visitor_df['Date'], visitor_df['Record Count'], label='Visitor', marker='x', linestyle='-', color=visitor_color)

    # Formatting the x-axis to display the month
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

    plt.xlabel('Date')
    plt.ylabel('Completed Surveys')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.ylim(0, 120)
    plt.show()

In [4]:
frequency_widget = widgets.Dropdown(
    options=['Weekly', 'Monthly', 'Quarterly'],
    value='Weekly',
    description='Frequency:',
)

In [5]:
# Function to update visualizations
def update_visualizations(frequency):
    # Load and preprocess the data (to be implemented)
    df = pd.read_csv('cleaned_survey.csv')

    # Plot run charts
    plot_run_charts(df, frequency)

In [6]:
# Display the interactive widget
interact_manual(update_visualizations, frequency=frequency_widget)

interactive(children=(Dropdown(description='Frequency:', options=('Weekly', 'Monthly', 'Quarterly'), value='We…

<function __main__.update_visualizations(frequency)>